<a href="https://colab.research.google.com/github/Hali57/FUTURE_ML_03/blob/main/Customer_Support_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

# Load the data -

In [ ]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset")

# Check structure
print(dataset)
print(dataset["train"].column_names)
print(dataset["train"][0])  # View a sample


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/11.9k [00:00<?, ?B/s]

(…)t_Training_Dataset_27K_responses-v11.csv:   0%|          | 0.00/19.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26872 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['flags', 'instruction', 'category', 'intent', 'response'],
        num_rows: 26872
    })
})
['flags', 'instruction', 'category', 'intent', 'response']
{'flags': 'B', 'instruction': 'question about cancelling order {{Order Number}}', 'category': 'ORDER', 'intent': 'cancel_order', 'response': "I've understood you have a question regarding canceling order {{Order Number}}, and I'm here to provide you with the information you need. Please go ahead and ask your question, and I'll do my best to assist you."}


## Data Cleaning

In [ ]:
import pandas as pd
import re
# Convert to DataFrame
df = pd.DataFrame(dataset["train"])

# Keep only relevant columns
df = df[['instruction', 'response']]

# Remove placeholders like {{Order Number}}
df['instruction'] = df['instruction'].apply(lambda x: re.sub(r'\{\{.*?\}\}', '', x).strip())
df['response'] = df['response'].apply(lambda x: re.sub(r'\{\{.*?\}\}', '', x).strip())

# Lowercase text (optional)
df['instruction'] = df['instruction'].str.lower()
df['response'] = df['response'].str.lower()

# Check sample
print(df.head())

                                   instruction  \
0              question about cancelling order   
1    i have a question about cancelling oorder   
2               i need help cancelling puchase   
3                    i need to cancel purchase   
4  i cannot afford this order, cancel purchase   

                                            response  
0  i've understood you have a question regarding ...  
1  i've been informed that you have a question ab...  
2  i can sense that you're seeking assistance wit...  
3  i understood that you need assistance with can...  
4  i'm sensitive to the fact that you're facing f...  


In [ ]:
df.columns

Index(['instruction', 'response'], dtype='object')

##  Tokenizing The Dataset

In [ ]:
from transformers import T5Tokenizer
import pandas as pd

# Load T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")

# Function to tokenize text and return input_ids and attention_mask
def tokenize_text(text):
    if not isinstance(text, str):  # Ensure input is a string
        text = str(text)
    tokenized = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    return tokenized["input_ids"].tolist()[0], tokenized["attention_mask"].tolist()[0]

# Apply tokenization to instruction and response columns
df["instruction_input_ids"], df["instruction_attention_mask"] = zip(*df["instruction"].map(tokenize_text))
df["response_input_ids"], df["response_attention_mask"] = zip(*df["response"].map(tokenize_text))

# Drop original text columns to create the final tokenized DataFrame
df_tokenized = df.drop(columns=["instruction", "response"])




spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
# Verify
print(df_tokenized['instruction_input_ids'][0])

[822, 81, 9179, 697, 455, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## Split the Data

In [ ]:
from sklearn.model_selection import train_test_split

# Define train (80%), validation (10%), and test (10%) split sizes
train_df, temp_df = train_test_split(df_tokenized, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Print dataset sizes
print(f"Training set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Test set size: {len(test_df)}")


Training set size: 21497
Validation set size: 2687
Test set size: 2688


## Prepare the data for training

In [ ]:
# Function to rename and format dataset columns
from datasets import Dataset
def format_dataset(df):
    df = df.rename(columns={
        "instruction_input_ids": "input_ids",
        "instruction_attention_mask": "attention_mask",
        "response_input_ids": "labels"
    })
    return Dataset.from_pandas(df).remove_columns(["response_attention_mask"])

# Convert Pandas DataFrames to properly formatted Hugging Face Datasets
train_dataset = format_dataset(train_df)
val_dataset = format_dataset(val_df)
test_dataset = format_dataset(test_df)

## Load Pretrained T5ForConditionalGeneration Model
` Now, we load the  T5ForConditionalGeneration model since we want the model to generate answers `

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Load the T5 model
model = T5ForConditionalGeneration.from_pretrained("t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

## Fine Tune The Model

In [ ]:
from transformers import TrainingArguments
from transformers import Trainer

training_args = TrainingArguments(
    output_dir="./results",  # Directory to save model checkpoints
    overwrite_output_dir=True,  # Overwrite the output dir if needed
    per_device_train_batch_size=2,  # Reduce batch size to avoid memory issues
    per_device_eval_batch_size=2,
    num_train_epochs=3,  # Number of epochs
    save_strategy="epoch",  # Save model per epoch
    evaluation_strategy="epoch",  # Evaluate per epoch
    logging_dir="./logs",  # Directory for logs
    logging_steps=10,  # Log every 10 steps
    report_to="none",  # Avoid TensorBoard logging issues
)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-10-309b92d7b3c0>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.271200,0.324807
2,0.395300,0.292581
3,0.293200,0.284546


TrainOutput(global_step=32247, training_loss=0.4037630987650894, metrics={'train_runtime': 6602.6744, 'train_samples_per_second': 9.767, 'train_steps_per_second': 4.884, 'total_flos': 8728328118730752.0, 'train_loss': 0.4037630987650894, 'epoch': 3.0})

## Saving the model

In [ ]:
import torch

# Define the save path
save_directory = "./t5_small_customer_support"

# Save model and tokenizer
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"Model and tokenizer saved to {save_directory}")


Model and tokenizer saved to ./t5_small_customer_support


## Model Evaluation

In [ ]:
from transformers import Trainer

# Load the trained model if needed
model.eval()

test_results = trainer.evaluate(eval_dataset=test_dataset)
print("Test Results:", test_results)


Test Results: {'eval_loss': 0.28816381096839905, 'eval_runtime': 85.8882, 'eval_samples_per_second': 31.297, 'eval_steps_per_second': 15.648, 'epoch': 3.0}


### Result
`The model has achieved an evaluation loss of 0.288, which is quite low. This suggests that the model has learned well and generalizes effectively to the test data.`

## Testing the model

In [ ]:
def generate_clean_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512).to(model.device)
    output_ids = model.generate(**inputs, max_length=150, num_beams=5, early_stopping=True)
    raw_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Clean up unnecessary phrases (adjust as needed)
    unwanted_phrases = ["i'll do my best!", "i'm here to guide you through the process of",
                        "rest assured,", "i'll get right on it!", "thank you for reaching out!"]

    for phrase in unwanted_phrases:
        raw_output = raw_output.replace(phrase, "")

    return raw_output.strip()

# Test again
for prompt in test_prompts:
    response = generate_clean_response(prompt)
    print(f"**Q:** {prompt}\n**A:** {response}\n")


**Q:** How can I reset my password?
**A:** resetting your password. to reset your password, please follow these steps: 1. log in to your account on our website. 2. navigate to the "my account" or "profile" section. 3. look for the option that says "" and click on it. 4. you will be prompted to enter the email address associated with your account. 5. you should receive an email with instructions on how to reset your password. if you encounter any difficulties or have further questions, please don't hesitate to reach out. we're here to assist you every step of the way.

**Q:** What is the return policy?
**A:** resolving the issue. to provide you with the most accurate information, could you please provide me with the or? with this information, i'll be able to provide you with the most up-to-date information regarding your return policy.  i'm here to assist you every step of the way.

**Q:** How do I contact customer support?
**A:** contacting our customer support team. to connect with ou

## Result
` The model can respond well to customer queries although some missing data exists in its response.  More tuning is required.`

### Download the chatbot

In [ ]:
import shutil
from google.colab import files

shutil.make_archive('./t5_small_customer_support', 'zip', './t5_small_customer_support')

files.download('./t5_small_customer_support.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>